In [ ]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

# Import from src
from src import SERIES_DIR, TRAIN_CSV
from src.bricks import Preprocessor, Predictor
from src.models import UNet3DClassifier

In [ ]:
# Load training data
df_train = pd.read_csv(TRAIN_CSV)
print(f"Loaded {len(df_train)} series from training set")

# Inference Pipeline

This notebook demonstrates how to use the Predictor class from src.bricks for inference on DICOM series.

## Load Models

Load pre-trained models for each modality.

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load models (update paths to your actual model files)
model_dict = {}

# Example - adjust paths as needed
# model_dict['CTA'] = UNet3DClassifier(in_ch=1, base_ch=32).to(device)
# model_dict['CTA'].load_state_dict(torch.load("path/to/model_CTA.pth"))

# model_dict['MRA'] = UNet3DClassifier(in_ch=1, base_ch=32).to(device)
# model_dict['MRA'].load_state_dict(torch.load("path/to/model_MRA.pth"))

print(f"Loaded {len(model_dict)} models")

## Initialize Predictor

The Predictor class from src.bricks handles the complete inference pipeline.

# Create predictor with loaded models
predictor = Predictor(
    model_dict=model_dict,
    cube_size=(48, 48, 48),
    stride=(28, 28, 28),
    device=device
)

print("Predictor initialized")
print(predictor)

## Run Inference on a Single Series

In [ ]:
# Example: predict on a single series
# Get first series from training data
import os
first_series_uid = df_train['SeriesInstanceUID'].iloc[0]
series_path = os.path.join(SERIES_DIR, first_series_uid)

if os.path.exists(series_path):
    predictions = predictor.predict_series(series_path)
    print(f"Series: {first_series_uid}")
    print(f"Predictions shape: {predictions.shape}")
    print(f"Aneurysm probability: {predictions[13]:.4f}")
else:
    print(f"Series not found locally: {series_path}")

## Batch Evaluation

Evaluate on multiple series and compute metrics.

In [ ]:
# Prepare evaluation dataset
from src.config import ANEURYSM_POSITIONS

loc_cols = ANEURYSM_POSITIONS
cols_to_keep = ['SeriesInstanceUID'] + loc_cols + ['Aneurysm Present']

# Filter to only available local series
available_series = [d for d in os.listdir(SERIES_DIR) 
                   if os.path.isdir(os.path.join(SERIES_DIR, d))]

df_eval = df_train[df_train['SeriesInstanceUID'].isin(available_series)][cols_to_keep].copy()

print(f"Available series for evaluation: {len(df_eval)}")
print(f"With aneurysm: {df_eval['Aneurysm Present'].sum()}")
print(f"Without aneurysm: {(1 - df_eval['Aneurysm Present']).sum()}")

In [ ]:
# Run predictions on evaluation set
y_true = []
y_pred = []

for idx, row in tqdm(df_eval.iterrows(), total=len(df_eval), desc="Evaluating"):
    series_uid = row['SeriesInstanceUID']
    series_path = os.path.join(SERIES_DIR, series_uid)
    
    try:
        predictions = predictor.predict_series(series_path)
        y_true.append(row['Aneurysm Present'])
        y_pred.append(predictions[13])  # Global aneurysm probability
    except Exception as e:
        print(f"Error on {series_uid}: {e}")
        continue

# Compute metrics
if len(y_true) > 0:
    auc = roc_auc_score(y_true, y_pred)
    print(f"\nResults:")
    print(f"  Evaluated: {len(y_true)} series")
    print(f"  AUC: {auc:.4f}")
else:
    print("No predictions made")